In [ ]:
using Gadfly, LightGraphs, Combinatorics

In [ ]:
import Base.+
+(x::UInt8,y::UInt8) = x $ y 
isocc(v) = !isempty(v.rowval)
top(v) = v.rowval[end]

function persistent_homology(∂)
    _top = Dict{Int,Int}()
    R = SparseMatrixCSC{UInt8,Int64}[∂[:,i] for i=1:size(∂,2)]

    for j=1:length(R)
        while isocc(R[j]) 
            i=top(R[j])
            if haskey(_top,i)
                R[j] += R[_top[i]]
            else
                _top[i] = j
                break
            end
        end
    end
    R
end

function threshold!(a::Dict{Vector{Int},Float64},σ::Vector{Int})  
    n = length(σ)
    if n == 1
        a[σ] = 1e-5
    elseif n ==2
        a[σ] = D[σ...]
    else
        r = 0.0
        for δ in combinations(σ,n-1)
            r = max(r,a[δ])
        end
        a[σ] = r + 1e-5
    end       
end

In [ ]:
N = 3 # number of pts

D = [0 0.8 0.26;
    0.8 0 0.4;
    0.24 0.4 0]

a_max = 3.0 # maximum radius
dim_max = 2

In [ ]:
N = 30; t = linspace(0,2π,N); x = cos(t); y = sin(t)
data = hcat([x y]',[x+3.5 y]')

M = size(data, 2)
D = zeros(M, M)
for i=1:M, j=1:M
D[i, j] = norm(data[:,i] - data[:,j])
end

a_max = 2.0
dim_max = 2

In [ ]:
g = Graph(Int.(D .< a_max)) # gplot(g) |> display
cliques = maximal_cliques(g)
dim_max = min(dim_max, maximum(map(length,cliques))-1)

filtrations = Array{Array{Int64,1},1}()
for dim in 0:dim_max
    for clique in cliques
          append!(filtrations,collect(combinations(clique,dim+1)))
    end
end
pop!(filtrations)

a = Dict{Vector{Int},Float64}() # threshold radius of σs
for σ in filtrations
    threshold!(a, σ)
end

p = sortperm(map(x->a[x],filtrations))
filtrations = filtrations[p]
index = Dict{Vector{Int},Int}()
for (i,σ) in enumerate(filtrations)
    index[σ] = i
end
index

∂ = spzeros(UInt8,length(filtrations),length(filtrations))
for (j,σ) in enumerate(filtrations)
    n = length(σ)
    if n ==1; continue; end
    for δ in combinations(σ,n-1)
        ∂[index[δ],j] = one(UInt8)
    end
end
full(Int.(∂));

In [ ]:
R = persistent_homology(∂)
Int.(full(hcat(R...)));

In [ ]:
birth = Float64[]
death = Float64[]
degree = Int[]
for j = 1:length(R)
    if !isempty(R[j].rowval)
        i = R[j].rowval[end] # birth time
        push!(birth,a[filtrations[i]]+0.01*rand())
        push!(death,a[filtrations[j]]+0.01*rand())
        push!(degree, length(filtrations[i])-1) # degree
    end
end
degree;     

In [ ]:
dims = map(x->length(x)-1,filtrations)
C = hist(dims,(0:dim_max+1)-0.5)[2]
C = Dict(zip(0:dim_max,C))
B = Dict{Int,Int}()
Z = Dict{Int,Int}()
H = Dict{Int,Int}()
B[-1] = 0
for dim in 0:dim_max
    B[dim] = sum(degree .== dim)
    Z[dim] = C[dim] - B[dim-1]
    H[dim] = Z[dim] - B[dim]
end
H

In [ ]:
p = plot(layer(x = birth, y = death, color = map(string,degree), Geom.point),
layer(x = [0,a_max], ymin = [0,0], ymax = [0,a_max], Geom.ribbon),
Guide.xlabel("birth"), Guide.ylabel("death"), Guide.colorkey("Betti Number"))
